Personalized Advisory using AI

In [15]:
import json
import os
import faiss
import numpy as np
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import ChatPromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI
from sentence_transformers import SentenceTransformer
from dotenv import load_dotenv

In [16]:
load_dotenv()
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")

if not GOOGLE_API_KEY:
    raise ValueError("GOOGLE_API_KEY not found. Please set it in your .env file.")

In [17]:
local_embedder = SentenceTransformer("all-MiniLM-L6-v2")

In [34]:
llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-pro",
    temperature=0.4,
    google_api_key=GOOGLE_API_KEY
)

In [42]:
json_input = {
    "farmer": {
        "_id": "68c056c710e11c03ad5665fa",
        "name": "Priya Verma",
        "phoneNumber": "9988776655",
        "location": "Pune, Maharashtra",
        "createdAt": "2025-09-09T16:33:11.810Z",
        "__v": 0
    },
    "farms": [
        {
            "activities": [],
            "_id": "68c0582e10e11c03ad56660a",
            "farmer": "68c056c710e11c03ad5665fa",
            "sizeInAcres": 7.8,
            "soilType": "Sandy",
            "irrigation": "Sprinkler",
            "location": "Pune, Maharashtra",
            "crops": [
                "Rice",
                "Vegetables"
            ],
            "createdAt": "2025-09-09T16:39:10.892Z",
            "__v": 0
        }
    ],
    "reminders": [
        {
            "_id": "68c05a5810e11c03ad566619",
            "farmer": "68c056c710e11c03ad5665fa",
            "message": "Schedule pest control treatment next week",
            "dueDate": "2025-09-18T09:30:00.000Z",
            "createdAt": "2025-09-09T16:48:24.179Z",
            "__v": 0
        }
    ]
}

In [43]:
context_text = json.dumps(json_input, indent=2)

In [44]:
splitter = RecursiveCharacterTextSplitter(chunk_size=200, chunk_overlap=50)
chunks = splitter.split_text(context_text)

In [45]:
embeddings = [local_embedder.encode(chunk) for chunk in chunks]
index = faiss.IndexFlatL2(len(embeddings[0]))
index.add(np.array(embeddings).astype("float32"))


In [46]:
prompt_template = ChatPromptTemplate.from_template(
    """
You are an agricultural advisory assistant.
Use the given context to provide polite, friendly, and practical advice for the farmer.
Keep the answer short, clear, and actionable.

Context:
{context}

Farmer's Question: {question}

Advisory:
"""
)

In [47]:
def retrieve_answer(question: str):
    # Embed query
    query_emb = local_embedder.encode(question)
    # Only retrieve top 1 chunk instead of 2
    D, I = index.search(np.array([query_emb]).astype("float32"), k=1)
    retrieved_chunks = [chunks[i] for i in I[0]]
    context = "\n".join(retrieved_chunks)


    # Format final prompt
    prompt = prompt_template.format(context=context, question=question)

    # Gemini response
    response = llm.predict(prompt)
    return response

In [48]:
if __name__ == "__main__":
    question = "What should I do about spraying on my brinjal crop?"
    answer = retrieve_answer(question)
    print("Farmer Advisory:\n", answer)

Farmer Advisory:
 Hello! Happy to help with your brinjal crop in Pune.

To give you the best advice, we first need to know what problem you are trying to solve.

**1. Inspect Your Plants First:**
Please take a close look at your brinjal plants for any of these common signs:
*   **Wilting shoots** or **small holes in the fruit** (This could be Fruit and Shoot Borer).
*   **Curled, yellow, or sticky leaves** (Check under the leaves for small insects like aphids or whiteflies).
*   **White powdery spots** on the leaves (This might be Powdery Mildew).

**2. General Spraying Tips:**
*   **Best Time:** Always spray in the **early morning or late evening** when the sun is not strong.
*   **Weather:** Avoid spraying on windy or rainy days.
*   **Coverage:** Make sure to spray thoroughly, including the underside of the leaves.

Let me know what you find on your plants, and I can suggest a more specific and effective solution for you
